    Copyright 2022 Sebastian T. Overskott Github link: https://github.com/Overskott/Quevo

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.

# Quevo - Example project

This notebook shows how you may use the Quevo to find quantum circuits fittet to a 1D Von Neumann neigbourhood for cellular automata. This guide will go through the process of initialising the needed parameters, creating chromosomes generations and in the end simulationg them on the qiskit simulator to see the quantum results.

First we import the Quevo library. __Note!__ You need Qiskit for the library to work.

In [43]:
from Quevo import *

Then we set up the evovlers parameters. 
- **gates**: The number of quantum gates per circuit.
- **chromosomes**: The number of chromosomes or solutions you want in each generation.
- **generation**: The number of generations you want to evolve before terminating.
- **desired_chance_of_one**: A list of the eight probabilities for the eight CA initial states you want the circuit to generate. There are some example probability list commented. Remove the comment to test them!

In [44]:
gates = 5
chromosomes = 10
generations = 20
gate_types = ['cx', 'h', '%x']  # possible gates: # h, cx, x, y, z, swap, rzz, rxx, toffoli

desired_chance_of_one = [0.394221, 0.094721, 0.239492, 0.408455, 0.0, 0.730203, 0.915034, 1.0]

In [45]:
# Generate initial generation of chromosomes
init_gen = Generation(chromosomes, gates)
init_gen.create_initial_generation(gate_types)
init_gen.print_chromosomes()



Chromosomes: 
[1, 2, 1, 0, 0, 2, 0, 2, 0, 0, 2, 1, 2, 0, 0]
[1, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 0, 0, 2]
[0, 1, 2, 1, 0, 0, 2, 0, 1, 1, 2, 0, 2, 0, 2]
[0, 0, 2, 2, 0, 1, 1, 0, 0, 2, 2, 2, 2, 1, 1]
[0, 2, 0, 2, 0, 2, 2, 0, 2, 1, 0, 0, 1, 1, 0]
[1, 2, 2, 2, 1, 0, 2, 0, 0, 1, 1, 2, 2, 1, 1]
[0, 2, 1, 2, 2, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0]
[0, 2, 1, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 2]
[0, 1, 0, 2, 0, 1, 2, 1, 2, 1, 1, 1, 1, 1, 0]
[2, 1, 1, 0, 0, 1, 2, 2, 2, 1, 0, 2, 0, 2, 0]




Then we check all the chromosomes fitness and print the best one found.

In [46]:
init_gen.run_generation_diff(desired_chance_of_one)

print("Fitness for best chromosome: " + str(init_gen.get_best_fitness()) + "\n"
      + "Best chromosome: \n" + str(init_gen.get_best_chromosome()))
print("\n")

Fitness for best chromosome: 2.449348
Best chromosome: 
[0, 1, 2, 1, 0, 0, 2, 0, 1, 1, 2, 0, 2, 0, 2]




Before we can start the evolution, we need to declare some container values fopr the best performing chromosome.

In [36]:
# Final value placeholders
current_chromosome = init_gen.get_best_chromosome()
best_chromosome = current_chromosome
final_fitness = init_gen.get_best_fitness()

Then we create a loop for the number of generations, and every time find the best chromosome in the generation to be a parent for the next generation.

In [38]:
# Mutation loop
for gen in range(0, generations):

    # Mutate next generation of chromosomes
    next_gen = Generation(chromosomes, gates)
    next_gen.create_mutated_generation(current_chromosome)

    # Check every Chromosome's fitness
    next_gen.run_generation_diff(desired_chance_of_one)

    current_fitness = next_gen.get_best_fitness()
    current_chromosome = next_gen.get_best_chromosome()
    
    # Print generation best result
    print("Fitness for best mutated chromosome in mutation " + str(gen + 1) + ": "
          + str(current_fitness) + "\n"
          + "Best mutated chromosome:\n" + str(next_gen.get_best_chromosome()))
    print("\n")

    # Check if there is a new_list best chromosome

    if final_fitness > current_fitness:
        final_fitness = current_fitness
        best_chromosome = current_chromosome
        print("New best!")
        
        print("\n")
    if current_fitness < 0.01: # Stop if high enough fitness is reached
        break
    print("------------------------------------------------------------------------------")


Fitness for best mutated chromosome in mutation 1: 2.0789999999999997
Best mutated chromosome:
[1, 1, 0, 0, 0, 2, 2, 0, 2, 0, 1, 0, 1, 0, 1]


New best!


------------------------------------------------------------------------------
Fitness for best mutated chromosome in mutation 2: 2.041
Best mutated chromosome:
[1, 1, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 1, 0, 1]


New best!


------------------------------------------------------------------------------
Fitness for best mutated chromosome in mutation 3: 2.052
Best mutated chromosome:
[1, 1, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 2, 1, 2]


------------------------------------------------------------------------------
Fitness for best mutated chromosome in mutation 4: 2.047
Best mutated chromosome:
[1, 1, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 1, 2, 1]


------------------------------------------------------------------------------
Fitness for best mutated chromosome in mutation 5: 2.038
Best mutated chromosome:
[1, 1, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 0, 0, 1]

Last, let's print the result of the best found ciurcuit and visualize it.

In [39]:
print("Best fitness found: " + str(final_fitness))
print("Best chromosome found: " + str(best_chromosome))
print("\n")
circuit = Circuit(best_chromosome)
circuit.print_ca_outcomes(desired_chance_of_one)


circuit.generate_circuit()
circuit.draw()

Best fitness found: 2.019
Best chromosome found: [0, 0, 2, 1, 0, 2, 0, 2, 0, 1, 2, 1, 0, 0, 1]


Initial State | Desired outcome | Actual outcome  | Difference
[0, 0, 0]           0.5000           0.5030           0.0030
[0, 0, 1]           0.7000           0.4900           0.2100
[0, 1, 0]           0.4000           0.5270           0.1270
[0, 1, 1]           0.0000           0.5180           0.5180
[1, 0, 0]           0.2000           0.4930           0.2930
[1, 0, 1]           0.7000           0.4600           0.2400
[1, 1, 0]           0.1000           0.5080           0.4080
[1, 1, 1]           0.9000           0.4870           0.4130
Total difference: 2.2119999999999997
          ┌───┐┌───┐     ┌─┐
q_0: ──■──┤ H ├┤ X ├──■──┤M├
       │  └───┘└─┬─┘┌─┴─┐└╥┘
q_1: ──┼─────────┼──┤ X ├─╫─
     ┌─┴─┐       │  ├───┤ ║ 
q_2: ┤ X ├───────■──┤ H ├─╫─
     └───┘          └───┘ ║ 
c: 1/═════════════════════╩═
                          0 
